In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import libraries


In [ ]:
import torch
from torchvision.transforms import transforms
import cv2
from torch.utils.data import DataLoader
import pandas as pd
import torchvision
from torchvision.datasets import ImageFolder
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import linear
from torch.nn.modules.activation import ReLU
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, confusion_matrix

# Unzip data


In [ ]:
# unzip all dataset
!unzip '/content/drive/MyDrive/noise_classification_phase16_red_green_blue_and_positive_blury_augmented_with_replaced_data_including_RGB_negative_and_RGB_positive_data_51561_training_3985_validation.zip'

In [ ]:
## unzip all_train data
!unzip '/content/drive/MyDrive/training_data_all.zip' 

In [ ]:
## unzip all_test data
!unzip '/content/drive/MyDrive/testing_data_all.zip'

In [ ]:
test_n = os.listdir('/content/testing_data/negative')
len(test_n)

1508

In [ ]:
test_p = os.listdir('/content/testing_data/positive')
len(test_p)

977

In [ ]:
train_n = os.listdir('/content/training_data/negative')
len(train_n)

20742

In [ ]:
train_p = os.listdir('/content/training_data/positive')
len(train_p)

11983

# Prepare data

## Remove wrong dimensions

In [ ]:
## remove wrong dimensions from the negative trainset
path = '/content/training_data/negative/'
for i in train_n:
  a = cv2.imread(path+i)
  if a.shape == (256, 320, 3):
    os.remove(path +i)
print(len(train_n))  

21461


In [ ]:
## remove wrong dimensions from the positive trainset
path = '/content/training_data/positive/'
for i in train_p:
  a = cv2.imread(path+i)
  if a.shape == (256, 320, 3):
    os.remove(path +i)
print(len(train_p))  

12400


## Create train and validation data


In [ ]:
#Transforms

transformer=transforms.Compose([
    transforms.ToTensor()  #0-255 to 0-1, numpy to tensors
])

In [ ]:
# load the data with labels

# path of training data
train_path = '/content/training_data'
test_path = '/content/testing_data'

# load the train and test data with labels
train_data = ImageFolder(train_path ,transform=transformer)
test_data = ImageFolder(test_path, transform=transformer)

In [ ]:
# set a batch size for train_set and test_set

train = DataLoader(train_data, batch_size=16 , shuffle=True)
test = DataLoader(test_data , batch_size=16, shuffle=True)

In [ ]:
len(train_data)

In [ ]:
len(train)

# Model architecture

# BSET ARCHITECTURE


In [ ]:
# TYPE_2 4TH_MODEL[RE]

model = nn.Sequential(
    
    # convolution layer 1                     # input shape(16,3,320,256)
    nn.Conv2d(in_channels =3 ,out_channels= 16, kernel_size = 3, stride=1, padding=1),
    nn.BatchNorm2d(num_features=16),
    nn.LeakyReLU(),
    nn.MaxPool2d(kernel_size = 2), #(16,16,160,128)

    # convolution layer 2                  
    nn.Conv2d(in_channels = 16 ,out_channels= 32, kernel_size = 3, stride=1, padding=1),  
    nn.BatchNorm2d(num_features=32),
    nn.LeakyReLU(), 
    nn.MaxPool2d(kernel_size = 2),   # output shape after max pooling  (16,32,80,64)

    # convolution layer 3                    
    nn.Conv2d(in_channels = 32 ,out_channels= 64, kernel_size = 3, stride=1, padding=1),  
    nn.BatchNorm2d(num_features=64),
    nn.LeakyReLU(),  
    nn.MaxPool2d(kernel_size = 2),   # output shape after max pooling  (16,64,40,32)

    # convolutional layer 4            
    nn.Conv2d(in_channels = 64 ,out_channels= 128, kernel_size = 3, stride=1, padding=1),   
    nn.BatchNorm2d(num_features=128),
    nn.LeakyReLU(),   
    nn.MaxPool2d(kernel_size = 2),   # output shape after max pooling  (16,128,20,16)    
    
    # convolutional layer 5             
    nn.Conv2d(in_channels = 128 ,out_channels= 256, kernel_size = 3, stride=1, padding=1),  
    nn.BatchNorm2d(num_features=256),
    nn.LeakyReLU(), 
    nn.MaxPool2d(kernel_size = 2),   # output shape after max pooling  (16,256,10,8)
    # flatten layer
    nn.Flatten(),
    
    # dropout
    nn.Dropout(p=0.2),

    # fully connected layer
    nn.Linear(in_features=256*10*8,out_features=250),
    nn.LeakyReLU(), 
    nn.Dropout(p=0.2),
    nn.Linear(in_features=250,out_features=50),
    nn.LeakyReLU(), 
    nn.Dropout(p=0.2),
    nn.Linear(in_features=50,out_features=1),
    nn.Sigmoid()
)

In [ ]:
# optimizer and loss function

lossfn = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)

# Enable GPU and model summary

In [ ]:
# enable GPU

device = 'cuda'
model.to(device)

In [ ]:
from torchsummary import summary

print(summary(model, (3,320,256)))

# Model training and validation

In [ ]:
# Save checkpoint if a new best is achieved
# Save checkpoint if test loss is minimum

# Save checkpoint if a new best is achieved
def best_save_checkpoint(state, is_best, filename):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("")

# Save checkpoint if test loss is minimum
def min_save_checkpoint(state, is_min, filename):
    if is_min:
        print ("")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("")
      

In [ ]:
# final code with graph
# save best accuracy 
# save minimum loss
# save train test accuracy and loss
# code for training and evalution
# code for plot between training and testing accuracy
# code for plot between training and testing loss

def train_model(model,lossfn ,optimizer,epochs=100):
  min_loss = 15.00
  best_acc = 0.0
  inter = 5
  training_acc = []
  training_loss = []
  testing_acc =[]
  testing_loss = []
  ## empty dict for appending acc and loss
  dic ={'epoch': [] ,'training_acc': [], 'training_loss': [],'testing_acc':
                    [],'testing_loss':[]}
  
  for epoch in range(1,epochs+1):
    train_loss = 0.0
    train_acc = 0.0
    
    model.train()

    for img ,label in train:
      img = img.cuda()
      label = label.cuda()

      pred = model(img)
      label = label.unsqueeze(1)
      label = label.float()

      loss = lossfn(pred, label)
           
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
         
      train_loss += loss.item()
      train_acc += ((pred > 0.5) == label).sum().item()
      

    training_acc.append(train_acc/len(train_data))
    training_loss.append(train_loss/len(train))

    # append per epoch train acc and loss
    dic['training_acc'].append(train_acc/len(train_data))
    dic['training_loss'].append(train_loss/len(train))
    dic['epoch'].append(epoch)
    # dict to dataframe
    df = pd.DataFrame.from_dict(dic, orient='index') 
    df = df.transpose()
    # convert df to csv and save csv
    df.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/save_acc_and_loss.csv',index=False)

    model.eval()

    test_loss = 0.0
    test_acc = 0.0

    with torch.no_grad():
      for img, label in test:
        img = img.cuda()
        label = label.cuda()

        pred = model(img)
        label = label.unsqueeze(1)
        label = label.float()
        loss = lossfn(pred,label)

        test_loss += loss.item()

        test_acc += ((pred > 0.5) == label).sum().item()
        

    testing_acc.append(test_acc/len(test_data))
    testing_loss.append(test_loss/len(test))

    # append per epoch test acc and loss
    dic['testing_acc'].append(test_acc/len(test_data))
    dic['testing_loss'].append(test_loss/len(test))
    # dict to dataframe
    df = pd.DataFrame.from_dict(dic, orient='index') 
    df = df.transpose()
    # convert df to csv and save csv
    df.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/save_acc_and_loss.csv',index=False)


    # plot training and testing accuracy
    plt.title('training and testing accuracy VS epochs')
    plt.figure()
    plt.plot(range(epoch), training_acc, label = 'training accuracy' )
    plt.plot(range(epoch), testing_acc, label = 'testing accuracy')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('training VS testing accuracy')
    plt.savefig('/content/training and testing accuracy graph_'+ str(epoch)+'.jpg')

    # plot training and testing loss
    plt.title('training and testing loss VS epochs')
    plt.figure()
    plt.plot(range(epoch), training_loss, label = 'training loss')
    plt.plot(range(epoch), testing_loss, label = 'testing loss')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('training VS testing loss')
    plt.savefig('/content/training and testing loss graph_'+ str(epoch)+'.jpg')
    

    ### save best test accuracy
    # get a accuracy of each epoch
    valid_acc_1 = test_acc/len(test_data)    
    # get boolian type base on current acc grater than the previous one
    # for make if condition true and false
    is_best = bool(valid_acc_1 > best_acc)
    # save previous acc in variable named best accuracy 
    best_acc = (max(valid_acc_1 , best_acc))

    #### save minimum test loss
    # get a loss of each epoch
    test_loss_1 = test_loss/len(test)
    # get boolian type base on current loss less than the previous one
    # for make if condition true and false
    is_min = bool(test_loss_1 < min_loss)
    # save previous acc in variable named best accuracy 
    min_loss = test_loss_1

    # print matrix
    print('epoch {} , training acc {:.4f} , testing acc {:.4f}, training loss {:.4f} , testing loss {:.4f}'.format(
        epoch , 100*train_acc/len(train_data) , 100*test_acc/len(test_data), 100*train_loss/len(train), 100*test_loss/len(test)
    ))
    
    #### call save_checkpoint function 
    # save best test accuracy
    best_save_checkpoint({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'best_accuracy': best_acc
    }, is_best,'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_epoch_'+str(epoch)+'_best_test_accuracy_'+str(best_acc)+'.pth' ) 
    
    #### call min_save_checkpoint
    # save minimum test loss 
    min_save_checkpoint({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'min_test_loss': min_loss
    }, is_min,'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_epoch_'+str(epoch)+'_min_test_loss_'+str(min_loss)+'.pth' ) 

    #### save checkpoints at a interval
    if epoch % inter == 0:
      torch.save({
        'epoch': epoch,
        'state_dict': model.state_dict()
      },'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_at_interval_epoch_'+str(epoch)+'.pth' ) 

    # plot training and testing accuracy
  plt.title('training and testing accuracy VS epochs')
  plt.plot(range(epochs), training_acc , label = 'training accuracy')
  plt.plot(range(epochs), testing_acc , label = 'testing accuracy')
  plt.legend()
  plt.xlabel('epochs')
  plt.ylabel('training VS testing accuracy')
  plt.savefig('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/training and testing accuracy graph.jpg')
  plt.show()

  # plot training and testing loss
  plt.title('training and testing loss VS epochs')
  plt.plot(range(epochs), training_loss , label = 'training loss')
  plt.plot(range(epochs), testing_loss , label = 'testing loss')
  plt.legend()
  plt.xlabel('epochs')
  plt.ylabel('training VS testing loss')
  plt.savefig('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/training and testing loss graph.jpg')
  plt.show()
 

In [ ]:
# call the train model function
train_model(model,lossfn = nn.BCELoss() , optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4))

# Plot accuracy and loss graph on both training and validation

In [ ]:
## plot and save graph of saved acc and loss

# read csv file
data = pd.read_csv('/content/drive/MyDrive/new_training/save_acc_and_loss_new.csv')

# plot training and testing accuracy
plt.title('training and testing accuracy VS epochs')
plt.plot(range(1,38),data['training_acc'] , label = 'training accuracy')
plt.plot(range(1,38), data['testing_acc'] , label = 'testing accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('training VS testing accuracy')
plt.savefig('/content/drive/MyDrive/new_training/training and testing accuracy graph.jpg')
plt.show()

# plot training and testing loss
plt.title('training and testing loss VS epochs')
plt.plot(range(1,38), data['training_loss'] , label = 'training loss')
plt.plot(range(1,38), data['testing_loss'] , label = 'testing loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('training VS testing loss')
plt.savefig('/content/drive/MyDrive/new_training/training and testing loss graph.jpg')
plt.show()
plt.close()

# Resuming training

## Load last epoch for retraining

In [ ]:
# function for loading model's weights

def load_model(filepath):
  checkpoints = torch.load(filepath)
  model.load_state_dict(checkpoints['state_dict'])
  print("=> loaded checkpoint '{}')".format(filepath))

In [ ]:
# call the load_model
load_model('/content/drive/MyDrive/model_at_interval_epoch_50_.pth')

## Initialize model, loss function and optimizer

In [ ]:
### for resuming training first intialize model, loss function and optimizer
model = model()
# optimizer and loss fuction
lossfn = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)

## Resuming training

In [ ]:

def train_model(model,lossfn ,optimizer,epochs=100):
  min_loss = 15.00
  best_acc = 0.0
  inter = 5
  training_acc = []
  training_loss = []
  testing_acc =[]
  testing_loss = []
  ## empty dict for appending acc and loss
  dic ={'epoch': [] ,'training_acc': [], 'training_loss': [],'testing_acc':
                    [],'testing_loss':[]}
  
  for epoch in range(1,epochs+1):
    train_loss = 0.0
    train_acc = 0.0
    
    model.train()

    for img ,label in train:
      img = img.cuda()
      label = label.cuda()

      pred = model(img)
      label = label.unsqueeze(1)
      label = label.float()

      loss = lossfn(pred, label)
           
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
         
      train_loss += loss.item()
      train_acc += ((pred > 0.5) == label).sum().item()
      

    training_acc.append(train_acc/len(train_data))
    training_loss.append(train_loss/len(train))

    # append per epoch train acc and loss
    dic['training_acc'].append(train_acc/len(train_data))
    dic['training_loss'].append(train_loss/len(train))
    dic['epoch'].append(epoch)
    # dict to dataframe
    df = pd.DataFrame.from_dict(dic, orient='index') 
    df = df.transpose()
    # convert df to csv and save csv
    df.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/save_acc_and_loss.csv',index=False)

    model.eval()

    test_loss = 0.0
    test_acc = 0.0

    with torch.no_grad():
      for img, label in test:
        img = img.cuda()
        label = label.cuda()

        pred = model(img)
        label = label.unsqueeze(1)
        label = label.float()
        loss = lossfn(pred,label)

        test_loss += loss.item()

        test_acc += ((pred > 0.5) == label).sum().item()
        

    testing_acc.append(test_acc/len(test_data))
    testing_loss.append(test_loss/len(test))

    # append per epoch test acc and loss
    dic['testing_acc'].append(test_acc/len(test_data))
    dic['testing_loss'].append(test_loss/len(test))
    # dict to dataframe
    df = pd.DataFrame.from_dict(dic, orient='index') 
    df = df.transpose()
    # convert df to csv and save csv
    df.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/save_acc_and_loss.csv',index=False)


    # plot training and testing accuracy
    plt.title('training and testing accuracy VS epochs')
    plt.figure()
    plt.plot(range(epoch), training_acc, label = 'training accuracy' )
    plt.plot(range(epoch), testing_acc, label = 'testing accuracy')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('training VS testing accuracy')
    plt.savefig('/content/training and testing accuracy graph_'+ str(epoch)+'.jpg')

    # plot training and testing loss
    plt.title('training and testing loss VS epochs')
    plt.figure()
    plt.plot(range(epoch), training_loss, label = 'training loss')
    plt.plot(range(epoch), testing_loss, label = 'testing loss')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('training VS testing loss')
    plt.savefig('/content/training and testing loss graph_'+ str(epoch)+'.jpg')
    

    ### save best test accuracy
    # get a accuracy of each epoch
    valid_acc_1 = test_acc/len(test_data)    
    # get boolian type base on current acc grater than the previous one
    # for make if condition true and false
    is_best = bool(valid_acc_1 > best_acc)
    # save previous acc in variable named best accuracy 
    best_acc = (max(valid_acc_1 , best_acc))

    #### save minimum test loss
    # get a loss of each epoch
    test_loss_1 = test_loss/len(test)
    # get boolian type base on current loss less than the previous one
    # for make if condition true and false
    is_min = bool(test_loss_1 < min_loss)
    # save previous acc in variable named best accuracy 
    min_loss = test_loss_1

    # print matrix
    print('epoch {} , training acc {:.4f} , testing acc {:.4f}, training loss {:.4f} , testing loss {:.4f}'.format(
        epoch , 100*train_acc/len(train_data) , 100*test_acc/len(test_data), 100*train_loss/len(train), 100*test_loss/len(test)
    ))
    
    #### call save_checkpoint function 
    # save best test accuracy
    best_save_checkpoint({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'best_accuracy': best_acc
    }, is_best,'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_epoch_'+str(epoch)+'_best_test_accuracy_'+str(best_acc)+'.pth' ) 
    
    #### call min_save_checkpoint
    # save minimum test loss 
    min_save_checkpoint({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'min_test_loss': min_loss
    }, is_min,'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_epoch_'+str(epoch)+'_min_test_loss_'+str(min_loss)+'.pth' ) 

    #### save checkpoints at a interval
    if epoch % inter == 0:
      torch.save({
        'epoch': epoch,
        'state_dict': model.state_dict()
      },'/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/'+'model_at_interval_epoch_'+str(epoch)+'.pth' ) 

    # plot training and testing accuracy
  plt.title('training and testing accuracy VS epochs')
  plt.plot(range(epochs), training_acc , label = 'training accuracy')
  plt.plot(range(epochs), testing_acc , label = 'testing accuracy')
  plt.legend()
  plt.xlabel('epochs')
  plt.ylabel('training VS testing accuracy')
  plt.savefig('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/training and testing accuracy graph.jpg')
  plt.show()

  # plot training and testing loss
  plt.title('training and testing loss VS epochs')
  plt.plot(range(epochs), training_loss , label = 'training loss')
  plt.plot(range(epochs), testing_loss , label = 'testing loss')
  plt.legend()
  plt.xlabel('epochs')
  plt.ylabel('training VS testing loss')
  plt.savefig('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/training and testing loss graph.jpg')
  plt.show()
 

In [ ]:
# call the train model function
train_model(model,lossfn = nn.BCELoss() , optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4))

# Prediction on validation data

## Load the best epoch model 

In [ ]:
# prediction on this model
# function for loading best epoch model's weights

def load_model(filepath):
  checkpoints = torch.load(filepath)
  model.load_state_dict(checkpoints['state_dict'])
  print("=> loaded checkpoint '{}' (best epoch acco to accu is {})".format(filepath, checkpoints['epoch']))

In [ ]:
# enable GPU

device = 'cuda'
model.to(device)

In [ ]:
# call the load_model
load_model('/content/drive/MyDrive/model_epoch_65_min_test_loss_0.030830490661826797_second.pth')

## Prediction

In [ ]:
# list of negative images of validation set
negative = os.listdir('/content/all_pred_tiles/negative')
source = '/content/all_pred_tiles/negative/'

# negative pred
dic = {'images': [], 'true_label': [], 'prediction': []}
c = 0
with torch.no_grad():
  model.eval()
  for i in negative:
    dic['images'].append(i)
    dic['true_label'].append(c)
    image = cv2.imread(source+i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(image)
    image = image.cuda()
    image = image.unsqueeze(0)
    pred = model(image)
    if (pred > 0.5):
      pred = 1
    else:
      pred = 0
    dic['prediction'].append(pred)
    dataf = pd.DataFrame.from_dict(dic, orient='index') 
    dataf = dataf.transpose()
    dataf.to_csv('/content/drive/MyDrive/created_data_pred_4th_model/with_negative_pred_65_epoch.csv',index=False)  

In [ ]:
# list of positive images of validation set
positive = os.listdir('/content/all_pred_tiles/positive')
source_p = '/content/all_pred_tiles/positive/'

# positive pred
c = 1
dic = {'images': [], 'true_label': [], 'prediction': []}
with torch.no_grad():
  model.eval()
  for i in positive:
    dic['images'].append(i)
    dic['true_label'].append(c)
    image = cv2.imread(source_p+i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(image)
    image = image.cuda()
    image = image.unsqueeze(0)
    pred = model(image)
    if (pred > 0.5):
      pred = 1
    else:
      pred = 0
    dic['prediction'].append(pred)
    dataf = pd.DataFrame.from_dict(dic, orient='index') 
    dataf = dataf.transpose()
    dataf.to_csv('/content/drive/MyDrive/created_data_pred_4th_model/with_positive_pred_65_epoch.csv',index=False)

In [ ]:
# read pred csv
df_p = pd.read_csv('/content/drive/MyDrive/created_data_pred_4th_model/with_positive_pred_65_epoch.csv')
df_n = pd.read_csv('/content/drive/MyDrive/created_data_pred_4th_model/with_negative_pred_65_epoch.csv')

## append dataframe
all_pred = df_p.append(df_n)

## all_pred convert to csv
all_pred.to_csv('/content/drive/MyDrive/created_data_pred_4th_model/all_positive_and_negative_pred_65_epoch.csv', index=False)
predi = pd.read_csv('/content/drive/MyDrive/created_data_pred_4th_model/all_positive_and_negative_pred_65_epoch.csv')
predi

## Evalution matrix of validation dataset


In [ ]:
## evulation of epoch 65 model
# final best epoch on base architecture model 
# prediction on validation set

y_true =[]
y_pred = []
for true , pred in zip(predi['true_label'],predi['prediction']):
  y_true.append(true)
  y_pred.append(pred)
  
precision = precision_score(y_true,y_pred)
print('precison of model: ', 100* precision)

recall = recall_score(y_true,y_pred)
print('Recall of model: ',100*recall)

f1_sc = f1_score(y_true,y_pred)
print('F1_score is: ', 100*f1_sc)

accu = accuracy_score(y_true,y_pred)
print('Accuracy is: ', 100*accu)

matrix = confusion_matrix(y_true,y_pred)
print('Confusion matrix is: \n', matrix)

# Prediction on Unseen dataset

## Load the best epoch

In [ ]:
# prediction unseen data
# function for loading best epoch model's weights

def load_model(filepath):
  checkpoints = torch.load(filepath)
  model.load_state_dict(checkpoints['state_dict'])
  print("=> loaded checkpoint '{}' (best epoch acco to accu is {})".format(filepath, checkpoints['epoch']))

In [ ]:
# enable GPU

device = 'cuda'
model.to(device)

In [ ]:
# call the load_model
load_model('/content/drive/MyDrive/model_epoch_65_min_test_loss_0.030830490661826797_second.pth')

## Prediction

In [ ]:
## import unseen excel
df_unseen = pd.read_excel('/content/noise_classification_phase16_red_green_blue_and_positive_blury_augmented_with_replaced_data_including_RGB_negative_and_RGB_positive_data_51561_training_3985_validation/Positive_and_Negative_Image_List_for_Unseen_Data.xlsx')
df_unseen.drop(df_unseen.index[0], inplace=True)
df_unseen


In [ ]:
# prediction on unseendata
# prediction on negative image 
source = '/content/noise_classification_phase16_red_green_blue_and_positive_blury_augmented_with_replaced_data_including_RGB_negative_and_RGB_positive_data_51561_training_3985_validation/unseen_set/Negative_Images/'
c = 0
dic = {'images': [], 'true_label': [], 'prediction': []}
with torch.no_grad():
  model.eval()
  for i in df_unseen['Negative Images']:
    dic['images'].append(i)
    dic['true_label'].append(c)
    image = cv2.imread(source+i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(image)
    image = image.cuda()
    image = image.unsqueeze(0)
    pred = model(image)
    if (pred > 0.5):
      pred = 1
    else:
      pred = 0
    dic['prediction'].append(pred)
    dataf = pd.DataFrame.from_dict(dic, orient='index') 
    dataf = dataf.transpose()
    dataf.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/with_negative_prediction_file_unseendata_epoch_73.csv',index=False)

In [ ]:
# prediction on positive image 
# prediction on unseendata
source = '/content/noise_classification_phase16_red_green_blue_and_positive_blury_augmented_with_replaced_data_including_RGB_negative_and_RGB_positive_data_51561_training_3985_validation/unseen_set/Positive_Images/'
c = 1
dic = {'images': [], 'true_label': [], 'prediction': []}
with torch.no_grad():
  model.eval()
  for i in df_unseen['Positive Images']:
    dic['images'].append(i)
    dic['true_label'].append(c)
    image = cv2.imread(source+i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(image)
    image = image.cuda()
    image = image.unsqueeze(0)
    #print(image.shape)
    pred = model(image)
    if (pred > 0.5):
      pred = 1
    else:
      pred = 0
    dic['prediction'].append(pred)
    dataf = pd.DataFrame.from_dict(dic, orient='index') 
    dataf = dataf.transpose()
    dataf.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/with_positive_prediction_file_unseendata_epoch_73.csv',index=False)

In [ ]:
df_unseen_p = pd.read_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/with_positive_prediction_file_unseendata_epoch_73.csv')
df_unseen_n = pd.read_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/with_negative_prediction_file_unseendata_epoch_73.csv')

In [ ]:
df_all_unseen_data = df_unseen_p.append(df_unseen_n)
df_all_unseen_data

In [ ]:
df_all_unseen_data.to_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/all_unseen_data_predi_epoch_73.csv',index=False)
# import unseen data predi csv
unseen = pd.read_csv('/content/drive/MyDrive/TYPE_2_4TH_MODEL_RE/all_unseen_data_predi_epoch_73.csv')
unseen

## Evalution matrix of unseen data

In [ ]:
## evulation of epoch 65 model
# final best epoch on base architecture model 
# prediction on UNSEEN set

y_true =[]
y_pred = []
for true , pred in zip(unseen['true_label'],unseen['prediction']):
  y_true.append(true)
  y_pred.append(pred)
  
precision = precision_score(y_true,y_pred)
print('precison of model: ', 100* precision)

recall = recall_score(y_true,y_pred)
print('Recall of model: ',100*recall)

f1_sc = f1_score(y_true,y_pred)
print('F1_score is: ', 100*f1_sc)

accu = accuracy_score(y_true,y_pred)
print('Accuracy is: ', 100*accu)

matrix = confusion_matrix(y_true,y_pred)
print('Confusion matrix is: \n', matrix)